Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

#### First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  # deleting save to help gc free up memory
  del save
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformatting the data into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

# Reformatting individual datasets
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

# Making sure that we have what we expect
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First we describe the computation that we want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph.

* Then we can run the operations on this graph as many times as we want by calling `session.run()`, providing it outputs to fetch from the graph that get returned.

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Now we run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.878115
Training accuracy: 5.0%
Validation accuracy: 7.1%
Loss at step 100: 2.351887
Training accuracy: 72.2%
Validation accuracy: 71.5%
Loss at step 200: 1.890586
Training accuracy: 75.3%
Validation accuracy: 73.5%
Loss at step 300: 1.643757
Training accuracy: 76.3%
Validation accuracy: 74.2%
Loss at step 400: 1.475981
Training accuracy: 77.1%
Validation accuracy: 74.9%
Loss at step 500: 1.350340
Training accuracy: 77.5%
Validation accuracy: 75.5%
Loss at step 600: 1.251743
Training accuracy: 78.1%
Validation accuracy: 75.7%
Loss at step 700: 1.171734
Training accuracy: 78.7%
Validation accuracy: 75.9%
Loss at step 800: 1.105048
Training accuracy: 79.2%
Validation accuracy: 76.1%
Test accuracy: 81.9%


Now we switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `sesion.run()`.

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Next we run the stochastic gradient descent optimization

In [7]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 25.101683
Minibatch accuracy: 4.7%
Validation accuracy: 8.0%
Minibatch loss at step 1000: 1.213710
Minibatch accuracy: 77.3%
Validation accuracy: 77.4%
Minibatch loss at step 2000: 0.911494
Minibatch accuracy: 82.8%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.804622
Minibatch accuracy: 76.6%
Validation accuracy: 79.5%
Minibatch loss at step 4000: 0.894080
Minibatch accuracy: 74.2%
Validation accuracy: 79.5%
Minibatch loss at step 5000: 0.863392
Minibatch accuracy: 75.8%
Validation accuracy: 79.1%
Minibatch loss at step 6000: 0.776066
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 7000: 0.704931
Minibatch accuracy: 83.6%
Validation accuracy: 80.0%
Minibatch loss at step 8000: 0.644504
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 9000: 0.710898
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 10000: 0.732382
Minibatch accuracy: 82.0%
Validation accura

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [8]:
# Using a one hidden layer, 1024 nodes, RELU based neural network model for above system
# Defining the batch size for sgd
batch_size_p = 128
n_RELU = 1024

# Defining the computation graph
graph_p = tf.Graph()

with graph_p.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size_p, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size_p, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Defining the variables
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_RELU]))
  biases1 = tf.Variable(tf.zeros([n_RELU]))

  weights2 = tf.Variable(
    tf.truncated_normal([n_RELU, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  def NNout(inp_data):
    # RELU layer input
    h_in = tf.matmul(inp_data, weights1) + biases1
    # RELU operation
    h_out = tf.nn.relu(h_in)
    # Ouput from the model
    return tf.matmul(h_out, weights2) + biases2 

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(NNout(tf_train_dataset), tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(NNout(tf_train_dataset))
  valid_prediction = tf.nn.softmax(NNout(tf_valid_dataset))
  test_prediction = tf.nn.softmax(NNout(tf_test_dataset))

In [9]:
num_steps = 10001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph_p) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size_p) % (train_labels.shape[0] - batch_size_p)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size_p), :]
    batch_labels = train_labels[offset:(offset + batch_size_p), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 341.831665
Minibatch accuracy: 3.1%
Validation accuracy: 33.4%
Minibatch loss at step 1000: 11.863055
Minibatch accuracy: 83.6%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 3.269826
Minibatch accuracy: 85.9%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 4.215050
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 4000: 2.750014
Minibatch accuracy: 82.8%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 1.049438
Minibatch accuracy: 91.4%
Validation accuracy: 83.8%
Minibatch loss at step 6000: 0.561062
Minibatch accuracy: 93.0%
Validation accuracy: 84.4%
Minibatch loss at step 7000: 0.514428
Minibatch accuracy: 93.8%
Validation accuracy: 84.9%
Minibatch loss at step 8000: 2.800675
Minibatch accuracy: 91.4%
Validation accuracy: 83.2%
Minibatch loss at step 9000: 0.156410
Minibatch accuracy: 95.3%
Validation accuracy: 85.3%
Minibatch loss at step 10000: 0.290465
Minibatch accuracy: 95.3%
Validation acc

#### Test accuaracy for 3001 steps without NN 86.0% and with NN 86.6%
#### Test accuracy for 5001 steps without NN 86.6% and with NN 89.2%
#### Test accuracy for 10001 steps without NN 87.3% and with NN 90.9 %